In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import geodesic

In [7]:
data_dir = os.path.join(os.getcwd(), '../data')
raw_data_dir = os.path.join(data_dir, 'raw')
Food_Delivery_Data = os.path.join(raw_data_dir, 'Food_Delivery_Data.xlsx')


preprocessed_out_dir = os.path.join(data_dir, 'Preprocessed')
os.makedirs(preprocessed_out_dir, exist_ok=True)
preprocessed_saved_filepath = os.path.join(preprocessed_out_dir, 'preprocessed.pkl')

In [8]:
# Load the food delivery dataset
df = pd.read_excel(Food_Delivery_Data)

---

## 2. Data Pre processing & Feature Engineering

### Initial Data Preview
Let's examine the first few rows to understand our dataset structure and identify key features.


In [9]:
# Display first 5 rows to understand data structure
print("Dataset Preview:")
display(df.head())

print("\nDataset Info:")
print(f"• Total records: {len(df):,}")
print(f"• Total features: {len(df.columns)}")
print(f"• Memory usage: {df.memory_usage().sum() / 1024**2:.2f} MB")

Dataset Preview:


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min)
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,24
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,33
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,26
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,21
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,30



Dataset Info:
• Total records: 45,593
• Total features: 11
• Memory usage: 3.83 MB


### Statistical Summary & Feature Analysis

Understanding our features and their distributions:
- **Numeric Features**: `Delivery_person_Age`, `Delivery_person_Ratings`, `Time_taken(min)`
- **Categorical Features**: `Type_of_vehicle`, `Type_of_order` (require encoding)
- **Target Variable**: `Time_taken(min)` (what we want to predict)
- **Location Features**: Latitude/longitude coordinates (will calculate distances)


In [10]:
# Generate comprehensive statistical summary
print("Statistical Summary of Numeric Features:")
display(df.describe())

print("\nData Types Overview:")
display(df.dtypes.to_frame('Data Type'))

print("\nFeature Categories:")
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"• Numeric features ({len(numeric_cols)}): {numeric_cols}")
print(f"• Categorical features ({len(categorical_cols)}): {categorical_cols}")

Statistical Summary of Numeric Features:


,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Time_taken(min)
count,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000
mean,29.544075,4.632367,17.017729,70.231332,17.465186,70.845702,26.294607
std,5.696793,0.327708,8.185109,22.883647,7.335122,21.118812,9.383806
min,15.000000,1.000000,-30.905562,-88.366217,0.010000,0.010000,10.000000
25%,25.000000,4.600000,12.933284,73.170000,12.988453,73.280000,19.000000
50%,29.000000,4.700000,18.546947,75.898497,18.633934,76.002574,26.000000
75%,34.000000,4.800000,22.728163,78.044095,22.785049,78.107044,32.000000
max,50.000000,6.000000,30.914057,88.433452,31.054057,88.563452,54.000000



Data Types Overview:


,Data Type
ID,object
Delivery_person_ID,object
Delivery_person_Age,int64
Delivery_person_Ratings,float64
Restaurant_latitude,float64
Restaurant_longitude,float64
Delivery_location_latitude,float64
Delivery_location_longitude,float64
Type_of_order,object
Type_of_vehicle,object



Feature Categories:
• Numeric features (7): ['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude', 'Time_taken(min)']
• Categorical features (4): ['ID', 'Delivery_person_ID', 'Type_of_order', 'Type_of_vehicle']


- **Features which are numeric** - Delivery_person_Age, Delivery_person_Ratings, Time_taken(min)
- **Feature which are categorical and needs OHE** - Type_of_vehicle, Type_of_order
- **Target feature** - Time_taken(min)
- **Feature handling for location latitude, location longitude**


### Data Quality Assessment
Checking for missing values and data completeness before proceeding with analysis.


In [11]:
# Check for missing values across all columns
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

print("Missing Values Analysis:")
missing_summary = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing %': missing_percentage.round(2)
}).sort_values('Missing %', ascending=False)

display(missing_summary[missing_summary['Missing Count'] > 0])

if missing_summary['Missing Count'].sum() == 0:
    print("No missing values found - dataset is complete!")

Missing Values Analysis:


,Missing Count,Missing %


No missing values found - dataset is complete!


In [12]:
# Dataset dimensions and basic info
print(f"Dataset Dimensions: {df.shape}")
print(f"   • Rows (Records): {df.shape[0]:,}")
print(f"   • Columns (Features): {df.shape[1]}")

print(f"\nDataset Size Metrics:")
print(f"   • Total data points: {df.size:,}")
print(f"   • Estimated memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Dataset Dimensions: (45593, 11)
   • Rows (Records): 45,593
   • Columns (Features): 11

Dataset Size Metrics:
   • Total data points: 501,523
   • Estimated memory: 12.32 MB


In [13]:
# Analyze feature cardinality (unique values per column)
print("Feature Cardinality Analysis:")
uniqueness = df.nunique().sort_values(ascending=False)
display(uniqueness.to_frame('Unique Values'))

print(f"\nCardinality Insights:")
high_cardinality = uniqueness[uniqueness > 100].index.tolist()
low_cardinality = uniqueness[uniqueness <= 10].index.tolist()

print(f"   • High cardinality features (>100 unique): {high_cardinality}")
print(f"   • Low cardinality features (≤10 unique): {low_cardinality}")
print(f"   • Potential ID columns: {[col for col in df.columns if 'id' in col.lower() or 'ID' in col]}")

Feature Cardinality Analysis:


,Unique Values
ID,45355
Delivery_location_longitude,4373
Delivery_location_latitude,4373
Delivery_person_ID,1320
Restaurant_latitude,657
Restaurant_longitude,518
Time_taken(min),45
Delivery_person_Ratings,28
Delivery_person_Age,22
Type_of_order,4



Cardinality Insights:
   • High cardinality features (>100 unique): ['ID', 'Delivery_location_longitude', 'Delivery_location_latitude', 'Delivery_person_ID', 'Restaurant_latitude', 'Restaurant_longitude']
   • Low cardinality features (≤10 unique): ['Type_of_order', 'Type_of_vehicle']
   • Potential ID columns: ['ID', 'Delivery_person_ID']


### Feature Engineering Strategy

**Key Transformations Planned:**
- **Delivery Person Analysis**: Extract patterns from `Delivery_person_ID` (number of deliveries, average rating)
- **Geographic Features**: Calculate geodesic distance between restaurant and delivery locations
- **Categorical Encoding**: Ordinal encoding for vehicle types, One-hot encoding for order types
- **Time Features**: Analyze delivery time patterns and create derived features

In [14]:
# Analyze delivery person patterns
delivery_person_stats = df['ID'].value_counts()
print("Delivery Person Analysis:")
print(f"   • Unique delivery persons: {df['ID'].nunique()}")
print(f"   • Average deliveries per person: {delivery_person_stats.mean():.1f}")
print(f"   • Max deliveries by single person: {delivery_person_stats.max()}")
print(f"   • Min deliveries by single person: {delivery_person_stats.min()}")

print(f"\nTop 10 Most Active Delivery Persons:")
print(delivery_person_stats.head(10).to_frame('Number of Deliveries'))

Delivery Person Analysis:
   • Unique delivery persons: 45355
   • Average deliveries per person: 1.0
   • Max deliveries by single person: 4
   • Min deliveries by single person: 1

Top 10 Most Active Delivery Persons:
            Number of Deliveries
ID                              
9000                           4
700                            4
6000                           4
7000                           4
800000                         3
7000000                        3
900                            3
6000000000                     3
400000000                      3
400                            3


### Geodesic Distance Implementation
Calculating accurate Earth-surface distances between restaurant and delivery locations using the haversine formula via geopy library.


### Pre-processing 

#### Why Geodesic Distance?
**Geodesic distance** represents the shortest path between two points on Earth's curved surface, measured along the surface rather than a straight line through space. This is crucial for delivery logistics because:

- **Accuracy**: Accounts for Earth's curvature, especially important over longer distances
- **Real-world relevance**: Reflects actual travel routes better than Euclidean distance  
- **Geographic precision**: Essential for location-based predictions in urban delivery scenarios


In [15]:
# Import geopy for accurate geographic calculations
from geopy.distance import geodesic
print("Geopy library loaded successfully for geodesic distance calculations")

Geopy library loaded successfully for geodesic distance calculations


In [16]:
def calculate_distance(row):
    """
    Calculate geodesic distance between restaurant and delivery locations.
    
    Args:
        row: DataFrame row containing location coordinates
        
    Returns:
        float: Distance in kilometers
    """
    # Extract coordinates
    restaurant_loc = (row['Restaurant_latitude'], row['Restaurant_longitude'])
    delivery_loc = (row['Delivery_location_latitude'], row['Delivery_location_longitude'])
    
    # Calculate geodesic distance
    distance_km = geodesic(restaurant_loc, delivery_loc).km
    return distance_km

print("Distance calculation function defined")

Distance calculation function defined


In [17]:
# Calculate geodesic distance for all records
print("Calculating geodesic distances...")
df['distance_geodesic_km'] = df.apply(calculate_distance, axis=1)
print("Distance calculation completed!")

# Verify calculation success
print(f"\nDistance Statistics:")
print(f"   • Mean distance: {df['distance_geodesic_km'].mean():.2f} km")
print(f"   • Median distance: {df['distance_geodesic_km'].median():.2f} km")
print(f"   • Max distance: {df['distance_geodesic_km'].max():.2f} km")
print(f"   • Min distance: {df['distance_geodesic_km'].min():.2f} km")

Calculating geodesic distances...
Distance calculation completed!

Distance Statistics:
   • Mean distance: 99.20 km
   • Median distance: 9.25 km
   • Max distance: 19709.58 km
   • Min distance: 1.46 km


In [18]:
# Display comprehensive distance statistics
print("Geodesic Distance Distribution:")
distance_stats = df['distance_geodesic_km'].describe()
display(distance_stats.to_frame('Distance (km)'))

# Check for potential outliers
q99 = df['distance_geodesic_km'].quantile(0.99)
outliers = (df['distance_geodesic_km'] > q99).sum()
print(f"\nPotential Distance Outliers (>99th percentile): {outliers} records")
print(f"   • 99th percentile threshold: {q99:.2f} km")

Geodesic Distance Distribution:


,Distance (km)
count,45593.000000
mean,99.198950
std,1099.925157
min,1.463837
25%,4.654185
50%,9.247024
75%,13.740932
max,19709.575543



Potential Distance Outliers (>99th percentile): 453 records
   • 99th percentile threshold: 20.94 km


## Feature Engineering 

### Categorical Variable Encoding Strategy
Implementing appropriate encoding techniques for categorical features based on their characteristics and relationships.

### Encoding categorical Variables

In [19]:
# Analyze vehicle type distribution
print("Vehicle Type Distribution:")
vehicle_counts = df['Type_of_vehicle'].value_counts()
display(vehicle_counts.to_frame('Count'))

print(f"\nVehicle Type Insights:")
print(f"   • Most common vehicle: {vehicle_counts.index[0]} ({vehicle_counts.iloc[0]:,} deliveries)")
print(f"   • Least common vehicle: {vehicle_counts.index[-1]} ({vehicle_counts.iloc[-1]:,} deliveries)")
print(f"   • Total vehicle types: {len(vehicle_counts)}")

Vehicle Type Distribution:


,Count
Type_of_vehicle,
motorcycle,26435
scooter,15276
electric_scooter,3814
bicycle,68



Vehicle Type Insights:
   • Most common vehicle: motorcycle  (26,435 deliveries)
   • Least common vehicle: bicycle  (68 deliveries)
   • Total vehicle types: 4


#### Vehicle Type Encoding: Ordinal Approach
**Strategy**: Use **Label Encoding** with custom ordinal mapping because vehicle types have a natural hierarchy based on speed and delivery capability:

- `bicycle` (0) → Slowest delivery method
- `electric_scooter` (1) → Moderate speed  
- `scooter` (2) → Faster than electric
- `motorcycle` (3) → Fastest delivery option

This preserves the inherent speed-based ordering that's crucial for delivery time prediction.

In [20]:
# Analyze order type distribution  
print("Order Type Distribution:")
order_counts = df['Type_of_order'].value_counts()
display(order_counts.to_frame('Count'))

print(f"\nOrder Type Insights:")
total_orders = len(df)
for order_type, count in order_counts.items():
    percentage = (count / total_orders) * 100
    print(f"   • {order_type}: {count:,} orders ({percentage:.1f}%)")

Order Type Distribution:


,Count
Type_of_order,
Snack,11533
Meal,11458
Drinks,11322
Buffet,11280



Order Type Insights:
   • Snack : 11,533 orders (25.3%)
   • Meal : 11,458 orders (25.1%)
   • Drinks : 11,322 orders (24.8%)
   • Buffet : 11,280 orders (24.7%)


#### Order Type Encoding: One-Hot Approach  
**Strategy**: Use **One-Hot Encoding** for order types because there's no clear ordinal relationship between "Snack," "Meal," "Drinks," and "Buffet."

Each order type affects delivery time differently based on:
- **Preparation complexity**: Meals vs snacks  
- **Package size**: Buffet vs drinks
- **Handling requirements**: Different logistics per type

One-hot encoding allows the model to learn independent effects for each category.


In [21]:
# Implement ordinal encoding for vehicle types based on delivery speed
print("Implementing Ordinal Encoding for Vehicle Types...")

# Define speed-based hierarchy mapping
ordinal_mapping = {
    'bicycle ': 0,           # Slowest: ~15-20 km/hr
    'electric_scooter ': 1,  # Moderate: ~25-30 km/hr
    'scooter ': 2,           # Fast: ~35-45 km/hr
    'motorcycle ': 3         # Fastest: 50+ km/hr
}

# Apply encoding
df['Type_of_vehicle_encoded'] = df['Type_of_vehicle'].map(ordinal_mapping)

print("Vehicle type encoding completed!")
print(f"\nEncoding Mapping Applied:")
for vehicle, code in ordinal_mapping.items():
    count = (df['Type_of_vehicle'] == vehicle).sum()
    print(f"   • {vehicle.strip()} → {code} ({count:,} records)")

Implementing Ordinal Encoding for Vehicle Types...
Vehicle type encoding completed!

Encoding Mapping Applied:
   • bicycle → 0 (68 records)
   • electric_scooter → 1 (3,814 records)
   • scooter → 2 (15,276 records)
   • motorcycle → 3 (26,435 records)


In [22]:
df['Type_of_vehicle_encoded'].value_counts()

Type_of_vehicle_encoded
3    26435
2    15276
1     3814
0       68
Name: count, dtype: int64

In [23]:
# Apply one-hot encoding to order types
print("Applying One-Hot Encoding to Order Types...")
original_shape = df.shape

# Create dummy variables (drop_first=True to avoid multicollinearity)
df = pd.get_dummies(df, columns=['Type_of_order'], drop_first=True)

print("One-hot encoding completed!")
print(f"\nEncoding Results:")
print(f"   • Original shape: {original_shape}")
print(f"   • New shape: {df.shape}")
print(f"   • New columns added: {df.shape[1] - original_shape[1]}")

# Show new columns created
new_columns = [col for col in df.columns if col.startswith('Type_of_order_')]
print(f"   • New order type columns: {new_columns}")

Applying One-Hot Encoding to Order Types...
One-hot encoding completed!

Encoding Results:
   • Original shape: (45593, 13)
   • New shape: (45593, 15)
   • New columns added: 2
   • New order type columns: ['Type_of_order_Drinks ', 'Type_of_order_Meal ', 'Type_of_order_Snack ']


In [24]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_vehicle,Time_taken(min),distance_geodesic_km,Type_of_vehicle_encoded,Type_of_order_Drinks,Type_of_order_Meal,Type_of_order_Snack
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,motorcycle,24,3.020737,3,False,False,True
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,scooter,33,20.143737,2,False,False,True
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,motorcycle,26,1.549693,3,True,False,False
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,motorcycle,21,7.774497,3,False,False,False
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,scooter,30,6.197898,2,False,False,True


In [25]:
# Remove redundant location columns after distance calculation
columns_to_drop = [
    'Restaurant_latitude', 'Restaurant_longitude',
    'Delivery_location_latitude', 'Delivery_location_longitude'
]

print(f"Removing redundant columns: {columns_to_drop}")
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

print("Cleanup completed!")
print(f"Updated dataset shape: {df.shape}")

Removing redundant columns: ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']
Cleanup completed!
Updated dataset shape: (45593, 11)


### Data Cleanup
Removing redundant location columns since we've already calculated the geodesic distance feature.

In [26]:
# Display final processed dataset structure
print("Final Processed Dataset Structure:")
print(df.head())

print(f"\nUpdated Feature Summary:")
print(f"   • Total features: {len(df.columns)}")
print(f"   • Numeric features: {len(df.select_dtypes(include=[np.number]).columns)}")
print(f"   • Object features: {len(df.select_dtypes(include=['object']).columns)}")
print(f"   • New engineered features: distance_geodesic_km, Type_of_vehicle_encoded, Type_of_order_*")

Final Processed Dataset Structure:
     ID Delivery_person_ID  Delivery_person_Age  Delivery_person_Ratings  \
0  4607     INDORES13DEL02                   37                      4.9   
1  B379     BANGRES18DEL02                   34                      4.5   
2  5D6D     BANGRES19DEL01                   23                      4.4   
3  7A6A    COIMBRES13DEL02                   38                      4.7   
4  70A2     CHENRES12DEL01                   32                      4.6   

  Type_of_vehicle  Time_taken(min)  distance_geodesic_km  \
0     motorcycle                24              3.020737   
1        scooter                33             20.143737   
2     motorcycle                26              1.549693   
3     motorcycle                21              7.774497   
4        scooter                30              6.197898   

   Type_of_vehicle_encoded  Type_of_order_Drinks   Type_of_order_Meal   \
0                        3                  False                False   

## Save the file locally

In [27]:
df.to_pickle(preprocessed_saved_filepath)